In [ ]:
#!/usr/bin/python3
# coding: utf-8
# sapporo

In [1]:
import codecs
from datetime import datetime as dt
import datetime
import sys
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
from urllib.request import urlretrieve
from cov19utils import download_if_needed

In [ ]:
#filename="kanagawa.geojson"
filename="aomori.geojson"
f=codecs.open(filename, "rb", encoding='utf-8')
j=json.load(f)
f.close()
f=codecs.open(filename.replace("geojson", "min.geojson"), "wb", encoding='utf-8')
f.write(json.dumps(j, ensure_ascii=False, separators=(',', ':')))
f.close()

In [18]:
#import geopandas as gpd
#https://plotly.com/python/network-graphs/
#https://plotly.com/python/scattermapbox/
#https://labs.mapbox.com/maki-icons/
df=pd.read_csv("csv/maps.csv", encoding='shift-jis', header=0)
#df = px.data.carshare()
#print(df)
fig = px.scatter_mapbox(
    df, lat="lat", lon="lon", color="patients",
    size="patients",
    color_continuous_scale=plotly.colors.sequential.Bluered,
    hover_name="name", 
    hover_data=["state", "category"],
    labels={'lat':'緯度', 'lon':'経度', 'state':'自治体', 'patients':'感染者数',
            'category':'形態', 'name':'名称', 'id':'ID'},
    size_max=20, zoom=8, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(template='plotly_dark')
fig.show()